# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fa00c9daca0>
├── 'a' --> tensor([[-0.8560,  1.7421, -0.5577],
│                   [-0.1771, -0.2366,  0.0304]])
└── 'x' --> <FastTreeValue 0x7fa00ca4fa30>
    └── 'c' --> tensor([[ 0.6761,  1.2141, -0.8292,  1.2611],
                        [-1.0266, -0.3765,  0.1702, -0.6013],
                        [-0.5274, -0.4058, -0.6577, -0.4860]])

In [4]:
t.a

tensor([[-0.8560,  1.7421, -0.5577],
        [-0.1771, -0.2366,  0.0304]])

In [5]:
%timeit t.a

79.8 ns ± 1.08 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fa00c9daca0>
├── 'a' --> tensor([[ 0.6718, -1.1520,  0.3386],
│                   [-0.2785,  1.3329,  0.7346]])
└── 'x' --> <FastTreeValue 0x7fa00ca4fa30>
    └── 'c' --> tensor([[ 0.6761,  1.2141, -0.8292,  1.2611],
                        [-1.0266, -0.3765,  0.1702, -0.6013],
                        [-0.5274, -0.4058, -0.6577, -0.4860]])

In [7]:
%timeit t.a = new_value

91.2 ns ± 0.453 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.8560,  1.7421, -0.5577],
               [-0.1771, -0.2366,  0.0304]]),
    x: Batch(
           c: tensor([[ 0.6761,  1.2141, -0.8292,  1.2611],
                      [-1.0266, -0.3765,  0.1702, -0.6013],
                      [-0.5274, -0.4058, -0.6577, -0.4860]]),
       ),
)

In [10]:
b.a

tensor([[-0.8560,  1.7421, -0.5577],
        [-0.1771, -0.2366,  0.0304]])

In [11]:
%timeit b.a

75.7 ns ± 0.862 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.3174, -1.2507,  2.1879],
               [ 1.4969,  1.1963, -0.2558]]),
    x: Batch(
           c: tensor([[ 0.6761,  1.2141, -0.8292,  1.2611],
                      [-1.0266, -0.3765,  0.1702, -0.6013],
                      [-0.5274, -0.4058, -0.6577, -0.4860]]),
       ),
)

In [13]:
%timeit b.a = new_value

666 ns ± 15.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.24 µs ± 16.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

14.6 µs ± 127 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

276 µs ± 9.15 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

265 µs ± 9.55 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f9f54c93f10>
├── 'a' --> tensor([[[-0.8560,  1.7421, -0.5577],
│                    [-0.1771, -0.2366,  0.0304]],
│           
│                   [[-0.8560,  1.7421, -0.5577],
│                    [-0.1771, -0.2366,  0.0304]],
│           
│                   [[-0.8560,  1.7421, -0.5577],
│                    [-0.1771, -0.2366,  0.0304]],
│           
│                   [[-0.8560,  1.7421, -0.5577],
│                    [-0.1771, -0.2366,  0.0304]],
│           
│                   [[-0.8560,  1.7421, -0.5577],
│                    [-0.1771, -0.2366,  0.0304]],
│           
│                   [[-0.8560,  1.7421, -0.5577],
│                    [-0.1771, -0.2366,  0.0304]],
│           
│                   [[-0.8560,  1.7421, -0.5577],
│                    [-0.1771, -0.2366,  0.0304]],
│           
│                   [[-0.8560,  1.7421, -0.5577],
│                    [-0.1771, -0.2366,  0.0304]]])
└── 'x' --> <FastTreeValue 0x7f9f54c93a60>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

48.9 µs ± 1.25 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f9f54c93b80>
├── 'a' --> tensor([[-0.8560,  1.7421, -0.5577],
│                   [-0.1771, -0.2366,  0.0304],
│                   [-0.8560,  1.7421, -0.5577],
│                   [-0.1771, -0.2366,  0.0304],
│                   [-0.8560,  1.7421, -0.5577],
│                   [-0.1771, -0.2366,  0.0304],
│                   [-0.8560,  1.7421, -0.5577],
│                   [-0.1771, -0.2366,  0.0304],
│                   [-0.8560,  1.7421, -0.5577],
│                   [-0.1771, -0.2366,  0.0304],
│                   [-0.8560,  1.7421, -0.5577],
│                   [-0.1771, -0.2366,  0.0304],
│                   [-0.8560,  1.7421, -0.5577],
│                   [-0.1771, -0.2366,  0.0304],
│                   [-0.8560,  1.7421, -0.5577],
│                   [-0.1771, -0.2366,  0.0304]])
└── 'x' --> <FastTreeValue 0x7f9f54ca20a0>
    └── 'c' --> tensor([[ 0.6761,  1.2141, -0.8292,  1.2611],
                        [-1.0266, -0.3765,  0.1702, -0.6013],
                 

In [23]:
%timeit t_cat(trees)

44.4 µs ± 665 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

82 µs ± 761 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.8560,  1.7421, -0.5577],
                [-0.1771, -0.2366,  0.0304]],
       
               [[-0.8560,  1.7421, -0.5577],
                [-0.1771, -0.2366,  0.0304]],
       
               [[-0.8560,  1.7421, -0.5577],
                [-0.1771, -0.2366,  0.0304]],
       
               [[-0.8560,  1.7421, -0.5577],
                [-0.1771, -0.2366,  0.0304]],
       
               [[-0.8560,  1.7421, -0.5577],
                [-0.1771, -0.2366,  0.0304]],
       
               [[-0.8560,  1.7421, -0.5577],
                [-0.1771, -0.2366,  0.0304]],
       
               [[-0.8560,  1.7421, -0.5577],
                [-0.1771, -0.2366,  0.0304]],
       
               [[-0.8560,  1.7421, -0.5577],
                [-0.1771, -0.2366,  0.0304]]]),
    x: Batch(
           c: tensor([[[ 0.6761,  1.2141, -0.8292,  1.2611],
                       [-1.0266, -0.3765,  0.1702, -0.6013],
                       [-0.5274, -0.4058, -0.6577, -0.4860]],
         

In [26]:
%timeit Batch.stack(batches)

112 µs ± 666 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.8560,  1.7421, -0.5577],
               [-0.1771, -0.2366,  0.0304],
               [-0.8560,  1.7421, -0.5577],
               [-0.1771, -0.2366,  0.0304],
               [-0.8560,  1.7421, -0.5577],
               [-0.1771, -0.2366,  0.0304],
               [-0.8560,  1.7421, -0.5577],
               [-0.1771, -0.2366,  0.0304],
               [-0.8560,  1.7421, -0.5577],
               [-0.1771, -0.2366,  0.0304],
               [-0.8560,  1.7421, -0.5577],
               [-0.1771, -0.2366,  0.0304],
               [-0.8560,  1.7421, -0.5577],
               [-0.1771, -0.2366,  0.0304],
               [-0.8560,  1.7421, -0.5577],
               [-0.1771, -0.2366,  0.0304]]),
    x: Batch(
           c: tensor([[ 0.6761,  1.2141, -0.8292,  1.2611],
                      [-1.0266, -0.3765,  0.1702, -0.6013],
                      [-0.5274, -0.4058, -0.6577, -0.4860],
                      [ 0.6761,  1.2141, -0.8292,  1.2611],
                      [-1.0266, -

In [28]:
%timeit Batch.cat(batches)

211 µs ± 3.35 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

571 µs ± 16.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
